# Here documents different version of PyTorch RL for simulation optimization

## Class BikeNet with R as dictionary

In [54]:
import seaborn as sb
import numpy as np
import random

In [57]:
a = np.random.exponential(2.0, 100000)
b = [random.expovariate(2.0) for x in range(100000)]
np.average(a), np.average(b)

(1.9997322401401938, 0.5016355238016227)

In [80]:
np.random.choice([1,2], 1, [0.1, 0.9])==2

array([ True])

In [ ]:
random.choices(list(range(5)), k=100)

In [48]:
import heapq
from heapq import heapqpop
from heapq import heapqpush
import random
import numpy as np

class BikeNet():
    def __init__(self, N, R, A, Q, repair, time_limit, start_position=0):
        self.N = N
        self.R = R
        self.A = A
        self.areas = list(range(A + 1))
        self.Q = Q
        self.repair = repair
        self.time_limit = time_limit
        self.car = start_position

        self.reset()

    def reset(self):
        self.T = 0
        self.carrier_position = self.car
        self.state = [int(self.N/self.A)] * self.A + [0]*self.A
        self.scheduler = []
        heapq.heapify(self.scheduler)
        for i in range(A):
            heappush(self.scheduler, [random.expovariate(self.R[i][0]), 1, i])
        heapq.heapify(self.scheduler)
        return np.array(self.state.copy())

    def step(self, action):
        rewards = 0
        # time for carrier to take the action and repair one bicycle
        t = (abs(self.carrier_position % (self.A - 1) - action % (self.A - 1)) + abs(
            self.carrier_position // (self.A - 1) - action // (self.A - 1))) * 0.5
        if self.state[action+self.A] > 0:
            self.state[self.carrier_position] += 1
            self.state[self.carrier_position + 1] -= 1
            t_cursor = self.T + t+self.repair
        else: t_cursor = self.T + t
            
        event = self.scheduler[0]
        self.T, kind, location = event[0], event[1], event[2]

        # update the atate of QN during the tansformation time
        while self.T < t_cursor:
            # 车到达
            if kind == 1:  
                self.state[location] += 1
            else:
                # 顾客到达
                if self.state[location] == 0:  #但没车
                    rewards -= 1
                else:
                    target = np.random.choice(self.areas, 1, p=self.Q[location])[0]
                    if target == self.A:  # 顾客到达，发现是坏车
                        self.state[location] -= 1
                        self.state[location + self.A] += 1
                        continue
                    else:  # 顾客到达，顺利骑行
                        self.state[location] -= 1
                        next_time = random.expovariate(self.R[location][0]) + self.T
                        if next_time <= self.time_limit:
                            heapq.heappush(self.scheduler, [next_time, 1, target])
                next_time = random.expovariate(self.R[location][1]) + self.T
                if next_time <= self.time_limit:
                    heapq.heappush(self.scheduler, [next_time, -1, location])
                    
            heapq.heappop(self.scheduler)
            if self.scheduler:
                event = self.scheduler[0]
                self.T, kind, location = event[0], event[1], event[2]
            else:
                break
        
        self.carrier_position = action
        self.T = t_cursor
        
        s_ = np.array(self.state.copy())

        if self.T <= self.time_limit and self.scheduler:
            return s_, rewards, 0
        else:
            return s_, rewards, 1




if __name__ == "__main__":
    # maze game
    random.seed(1)
    N = 80  # total number of bikes in the QN
    A = 4  # A for areas, indicates the number of areas and the action space
    R = {} #[customer_arrval, ride]
    for i in range(A): R[i] = [1.0, 0.5]
    Q = [np.random.rand(A) for i in range(A)]
    Q = [q / sum(q)*0.99 for q in Q]
    Q = [np.append(q, 0.01) for q in Q]
    t_repair = 0.5
    time_limit = 10
    start_position = 0
    env = BikeNet(N, R, A, Q, t_repair, time_limit, start_position)
    for i in range(10):
        print(env.step(np.random.randint(0, A)))
    env.reset()



(array([ 3, 23,  7, 15,  7,  2, 11,  2]), 0, 0)
(array([13, 15,  3, 17,  2,  5, 14,  3]), 0, 0)
(array([ 6, 17,  7, 20,  3,  3,  5,  3]), 0, 0)
(array([15, 17,  6, 21,  1,  3,  7,  0]), 0, 0)
(array([18, 18, 11, 14,  1,  1,  5,  1]), 0, 0)
(array([18, 20,  5, 19,  1,  0, 12,  0]), 0, 0)
(array([11, 17,  5, 20,  3,  1, 11,  3]), 0, 0)
(array([15, 11,  6, 23,  3,  4, 14,  0]), 0, 0)
(array([21,  5,  0, 21,  1,  9, 13,  1]), -2, 0)
(array([14, 20,  6, 19,  2,  1,  8,  3]), 0, 0)


## Class BikeNet with R as DataFrame

In [39]:
import heapq
import numpy as np

class BikeNet():
    def __init__(self, N, R, A, Q, repair, time_limit, start_position):
        self.N = N
        self.R = R
        self.A = A
        self.areas = list(range(A + 1))
        self.Q = Q
        self.repair = repair
        self.time_limit = time_limit
        self.car = start_position

        self.reset()

    def reset(self):
        self.T = 0
        self.carrier_position = self.car
        self.place = list(range(self.A))
        self.time = sorted([np.random.exponential(1.0 / self.R.loc[i].cus_arr) for i in self.place], reverse=True)
        self.type = [-1] * self.A
        self.state = [int(self.N / self.A)] * self.A + [0] * self.A
        self.scheduler = [[self.time[x], self.type[x], self.place[x]] for x in range(self.A)]
        heapq.heapify(self.scheduler)
        return np.array(self.state.copy())

    def step(self, action):
        rewards = 0
        # time for carrier to take the action and repair one bicycle
        t = (abs(self.carrier_position % (self.A - 1) - action % (self.A - 1)) + abs(
            self.carrier_position // (self.A - 1) - action // (self.A - 1))) / \
            self.R.loc[0].ride + self.repair
        t_cursor = self.T + t
        event = self.scheduler[0]
        self.T, kind, location = event[0], event[1], event[2]
        self.carrier_position = action

        # update the atate of QN during the tansformation time
        while self.T < t_cursor:
            if kind == 1:  # 车到达
                self.state[location] += 1
            else:
                if self.state[location] == 0:  # 顾客到达，但没车
                    rewards -= 1
                else:
                    target = np.random.choice(self.areas, 1, p=self.Q[location])[0]
                    if target == self.A:  # 顾客到达，发现是坏车
                        self.state[location] -= 1
                        self.state[location + self.A] += 1
                        continue
                    else:  # 顾客到达，顺利骑行
                        self.state[location] -= 1
                        next_time = np.random.exponential(1.0 / self.R.loc[location].ride) + self.T
                        if next_time <= self.time_limit:
                            heapq.heappush(self.scheduler, [next_time, 1, target])
                next_time = np.random.exponential(1.0 / self.R.loc[location].cus_arr) + self.T
                if next_time <= self.time_limit:
                    heapq.heappush(self.scheduler, [next_time, -1, location])
            heapq.heappop(self.scheduler)
            if self.scheduler:
                event = self.scheduler[0]
                self.T, kind, location = event[0], event[1], event[2]
            else:
                break

        self.T = t_cursor
        if self.state[self.carrier_position] > 0:
            self.state[self.carrier_position] -= 1
            self.state[self.carrier_position + self.A] += 1
        s_ = np.array(self.state.copy())

        if self.T <= self.time_limit and self.scheduler:
            return s_, rewards, 0
        else:
            return s_, rewards, 1


import numpy as np
import pandas as pd

if __name__ == "__main__":
    # maze game
    np.random.seed(1)
    N = 80  # total number of bikes in the QN
    A = 4  # A for areas, indicates the number of areas and the action space
    R = pd.DataFrame({'cus_arr': [1] * A, 'ride': [0.5] * A}, index=range(A))
    Q = [np.random.rand(A + 1) for i in range(A)]
    Q = [q / sum(q) for q in Q]
    # Q = [[0,0.9,0.1], [0.9,0,0.1]]
    t_repair = 5
    P = 0
    time_limit = 10
    start_position = 0
    env = BikeNet(N, R, A, Q, t_repair, time_limit, start_position)
    #     RL = Train(A, 2*A,
    #               learning_rate=0.01,
    #               reward_decay=0.9,
    #               e_greedy=0.9,
    #               replace_target_iter=200,
    #               memory_size=2000
    #               # output_graph=True
    #               )
    #     net = DQN(8,4)
    #     output = net(torch.randn(1,8))
    for i in range(10):
        print(env.step(np.random.randint(0, A)))
        env.reset()





(array([18, 18, 17, 18,  0,  2,  0,  0]), 0, 0)
(array([18, 18, 18, 17,  1,  2,  0,  1]), 0, 0)
(array([21, 19, 17, 14,  1,  1,  0,  1]), 0, 0)
(array([15, 14, 19, 14,  1,  3,  0,  1]), 0, 0)
(array([15, 10, 21, 12,  0,  9,  1,  2]), 0, 0)
(array([16, 15, 16, 16,  4,  3,  0,  0]), 0, 0)
(array([17, 19, 17, 15,  0,  2,  0,  0]), 0, 0)
(array([16, 13, 20, 18,  2,  5,  0,  0]), 0, 0)
(array([23, 18, 17, 13,  0,  1,  0,  2]), 0, 0)
(array([16, 18, 14, 16,  0,  6,  1,  1]), 0, 0)


In [11]:
a = list(range(100000000))
%timeit a[0]
%timeit a[-1]

40 ns ± 2.22 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
37.2 ns ± 1.39 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [61]:
a = torch.zeros(1, 100)
t = torch.eye(100)[:, 98]
b = np.array([0]*100)
c = [0]*100
%timeit a+t
%timeit b[98]+=1
%timeit c[98]+=1

2.95 µs ± 20.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
442 ns ± 3.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
82.9 ns ± 0.449 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## PyTorch Train

In [34]:
import pandas  as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


# np.random.seed(1)
# USE_CUDA = torch.cuda.is_available()


# Deep Q Network off-policy
class DQN(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(DQN, self).__init__()
        self.line1 = nn.Linear(num_inputs, 64)
        self.line2 = nn.Linear(64, 32)
        self.line3 = nn.Linear(32, num_outputs)

    def forward(self, x):
        x = F.relu(self.line1(x))
        x = F.relu(self.line2(x))
        x = self.line3(x)
        return x


class Train():
    def __init__(
            self,
            n_actions,
            n_features,
            learning_rate=0.01,
            reward_decay=0.9,
            e_greedy=0.9,
            replace_target_iter=300,
            memory_size=500,
            batch_size=32,
            e_greedy_increment=None,
            output_graph=False,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max

        # total learning step
        self.learn_step_counter = 0

        # initialize zero memory [s, a, r, s_]
        self.memory = np.zeros((self.memory_size, n_features * 2 + 2))

        # consist of [target_net, evaluate_net]
        self.target_net = DQN(n_features, n_actions)
        self.eval_net = DQN(n_features, n_actions)

        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.eval_net.parameters(), lr=self.lr)

    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition
        self.memory_counter += 1

    def choose_action(self, observation):
        # to have batch dimension when feed into tf placeholder
        observation = observation[np.newaxis, :]

        if np.random.uniform() < self.epsilon:
            # forward feed the observation and get q value for every actions
            
            action = np.argmax(actions_value)
        else:
            action = np.random.randint(0, self.n_actions)
        return action

    def learn(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            # self.sess.run(self.target_replace_op)
            self.target_net.load_state_dict(self.eval_net.state_dict())
            # print('\ntarget_params_replaced\n')

        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        self.s = Variable(torch.from_numpy(batch_memory[:, :self.n_features]).float(), requires_grad=True)
        self.a = Variable(torch.from_numpy(batch_memory[:, self.n_features]).long())
        self.r = Variable(torch.from_numpy(batch_memory[:, self.n_features + 1]).float(), requires_grad=True)
        self.s_ = Variable(torch.from_numpy(batch_memory[:, -self.n_features:]).float(), requires_grad=True)

        current_Q_values = self.eval_net(self.s).gather(1, self.a.unsqueeze(1)).view(-1)
        next_Q_values = self.target_net(self.s_).detach().max(1)[0]
        # Compute the target of the current Q values
        target_Q_values = self.r + (self.gamma * next_Q_values)
        # Compute Bellman error
        loss = self.criterion(target_Q_values, current_Q_values)
        #         # clip the bellman error between [-1 , 1]
        #         clipped_bellman_error = bellman_error.clamp(-1, 1)
        #         # Note: clipped_bellman_delta * -1 will be right gradient
        #         d_error = clipped_bellman_error * -1.0
        #         # Clear previous gradients before backward pass
        self.optimizer.zero_grad()
        # run backward pass
        loss.backward()

        # Perfom the update
        self.optimizer.step()

        # increasing epsilon
        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1


## Original version of Class BikeNet

In [4]:
import numpy as np
import pandas as pd

class Area():
    def __init__(self, n, a_id):
        self.a_id = a_id
        self.normal_bike = n
        self.broken_bike = 0

    def move(self):
        self.normal_bike -= 1
        self.broken_bike += 1
        
    def repair(self):
        self.normal_bike += 1
        self.broken_bike -= 1


def binaryInsert(target, events):
    for event in events:
        if event >= target[-1]:
            target.append(event)
        else:
            l, mid, r = 0, int(len(target) / 2), len(target) - 1
            while 1:
                if r - l == 1:
                    target.insert(r, event)
                    break
                else:
                    if event > target[mid]:
                        l = mid
                        mid = int((r + l) / 2)
                    else:
                        r = mid
                        mid = int((r + l) / 2)


class BikeNet():
    def __init__(self, N, R, A, Q, repair, time_limit, start_position):
        self.N = N
        self.R = R
        self.A = A
        self.Q = Q
        self.repair = repair
        self.time_limit = time_limit
        self.carrier_position = start_position
        self.reset()
        self.trans = {}

    def reset(self):
        # self.__init__(self.N, self.R, self.A, self.Q, self.P, self.time_limit)
        # stat, S = pd.DataFrame(columns=['type', 'place', 't']), 0
        # loss, L = pd.DataFrame(columns=['place', 't']), 0
        # broken, B = pd.DataFrame(columns=['place', 'ng', 'nb', 't']), 0

        # initiation of instances of Area and scheduler
        self.T = 0
        self.scheduler = []
        self.a = []  # list of instances of areas
        self.s = np.array([(self.N / self.A) if i % 2 == 0 else 0 for i in range(2 * self.A)])
        for i in range(A):
            self.a.append(Area(self.N / self.A, i))
            self.scheduler.append([np.random.exponential(1 / self.R.loc[i].cus_arr), 1, self.a[i]])
        self.scheduler.sort()

        return self.s.copy()

    def step(self, action):
        # time for carrier to take the action and repair one bicycle
        t = (np.abs(self.carrier_position % 3 - action % 3) + np.abs(self.carrier_position // 3 - action // 3)) / \
            self.R.loc[0].ride + self.repair
        t_cursor = self.T + t
        self.carrier_position = action
        reward = 0

        # update the atate of QN during the tansformation time
        while self.T < t_cursor:
            self.T = self.scheduler[0][0]
            if self.scheduler[0][1] == 1:
                # stat.loc[S], S = [scheduler[0][1], scheduler[0][2].a_id, T], S+1
                if self.scheduler[0][2].normal_bike == 0:
                    # this is a loss
                    reward -= 1
                    # loss.loc[L], L = [scheduler[0][2].a_id, self.T], L+1
                    event = [self.T + np.random.exponential(1 / self.R.loc[self.scheduler[0][2].a_id].cus_arr), 1,
                             self.scheduler[0][2]]
                    binaryInsert(self.scheduler, [event])
                else:
                    target = np.random.choice(np.arange(self.A + 1), 1, p=self.Q[self.scheduler[0][2].a_id])
                    if target == self.A:
                        # broken.loc[B], B = [self.scheduler[0][2].a_id, self.scheduler[0][2].normal_bike, self.scheduler[0][2].broken_bike, T], B+1
                        self.scheduler[0][2].move()
                        self.s[self.scheduler[0][2].a_id * 2], self.s[self.scheduler[0][2].a_id * 2 + 1] = \
                        self.scheduler[0][2].normal_bike, self.scheduler[0][2].broken_bike
                        continue
                    else:
                        self.scheduler[0][2].normal_bike -= 1
                        self.s[self.scheduler[0][2].a_id * 2] -= 1
                        event1 = [self.T + np.random.exponential(1 / self.R.loc[self.scheduler[0][2].a_id].ride), 2,
                                  self.a[target[0]]]
                        event2 = [self.T + np.random.exponential(1 / self.R.loc[self.scheduler[0][2].a_id].cus_arr), 1,
                                  self.scheduler[0][2]]
                        binaryInsert(self.scheduler, [event1, event2])
            else:
                # stat.loc[S], S = [scheduler[0][1], scheduler[0][2].a_id, T], S+1
                self.scheduler[0][2].normal_bike += 1
                self.s[self.scheduler[0][2].a_id * 2] += 1
            self.scheduler.pop(0)
        
        self.a[action].repair()
        s_ = self.s.copy()

        self.T = t_cursor
        if self.T < self.time_limit:
            return s_, reward, 0
        else:
            return s_, reward, 1

## test BikeNet

In [5]:
import numpy as np
import pandas as pd
if __name__ == "__main__":
    # maze game
    np.random.seed(1)
    N = 360 #total number of bikes in the QN
    A = 9 #A for areas, indicates the number of areas and the action space
    R = pd.DataFrame({'cus_arr': [5] * A, 'ride': [10] * A}, index=range(A))
    Q = [np.random.rand(A+1) for i in range(A)]
    Q = [q / sum(q) for q in Q]
    #Q = [[0,0.9,0.1], [0.9,0,0.1]]
    t_repair = 5
    P = 0
    time_limit = 10
    start_position = 0
    env = BikeNet(N, R, A, Q, t_repair, time_limit, start_position)
#     RL = Train(A, 2*A,
#               learning_rate=0.01,
#               reward_decay=0.9,
#               e_greedy=0.9,
#               replace_target_iter=200,
#               memory_size=2000
#               # output_graph=True
#               )
#     net = DQN(8,4)
#     output = net(torch.randn(1,8))
    for i in range(10):
        #print(env.time)
        print(env.step(np.random.randint(0, 9)))
        env.reset()


(array([46.,  3., 53.,  1.,  8., 11., 42.,  1., 36.,  0., 36.,  7., 44.,
        3., 23.,  5., 35.,  0.]), 0, 0)
(array([32.,  9., 49.,  0., 32.,  4., 48.,  3., 29.,  1., 27.,  5., 37.,
        7., 24.,  3., 46.,  1.]), 0, 0)
(array([38.,  4., 51.,  2.,  0., 15., 50.,  0., 38.,  0., 40.,  4., 42.,
        5., 26.,  5., 36.,  1.]), -2, 0)
(array([26., 10., 47.,  4., 25.,  6., 55.,  4., 29.,  0., 26.,  9., 41.,
        4., 34.,  5., 29.,  3.]), 0, 0)
(array([41.,  2., 41.,  2., 25.,  4., 37.,  8., 23.,  3., 35.,  2., 51.,
        1., 44.,  2., 34.,  3.]), 0, 0)
(array([45.,  3., 45.,  1., 20.,  9., 27.,  5., 46.,  0., 34.,  5., 46.,
        3., 30.,  2., 31.,  3.]), 0, 0)
(array([43.,  1., 57.,  0., 22.,  2., 42.,  2., 26.,  1., 29.,  6., 39.,
        4., 28.,  5., 45.,  1.]), 0, 0)
(array([36.,  6., 44.,  0., 12., 18., 57.,  0., 38.,  0., 43.,  2., 26.,
        7., 32.,  1., 33.,  2.]), 0, 0)
(array([33.,  8., 49.,  2., 32.,  6., 37.,  1., 26.,  1., 25.,  7., 37.,
        6., 36.,  4., 

In [54]:
output

tensor([[ 0.0423, -0.0537, -0.1885,  0.0527]], grad_fn=<AddmmBackward>)

In [53]:
np.argmax(output.detach().numpy())

3

## Main Function

In [ ]:
def simulate():
    n_episodes = 3
    result = []
    for episode in range(n_episodes):
        sum_r = 0
        step = 0
        # initial observation
        observation = env.reset()
        # observation = np.array(int(N/A)) #devide all the normal bikes to all the areas evenly at the beginning

        while True:
            # fresh env
            # env.render()

            # RL choose action based on observation
            action = RL.choose_action(observation)
            # action = (action+1)%9
            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)
            RL.store_transition(observation, action, reward, observation_)

            if (step > 200) and (step % 5 == 0):
                RL.learn()
            #RL.learn()

            # swap observation
            observation = observation_

            # break while loop when end of this episode
            if done:
                break
            step += 1
            sum_r += reward

        result.append([episode, sum_r, env.T])

    # end of game
    print('learning over')
    return result


if __name__ == "__main__":
    # maze game
    np.random.seed(10)
    N = 80  # total number of bikes in the QN
    A = 4  # A for areas, indicates the number of areas and the action space
    R = pd.DataFrame({'cus_arr': [1] * A, 'ride': [0.5] * A}, index=range(A))
    Q = [np.random.rand(A) for i in range(A)]
    Q = [q / sum(q) * 0.99 for q in Q]
    Q = [np.append(q, 0.01) for q in Q]
    # Q = [[0,0.9,0.1], [0.9,0,0.1]]
    t_repair = 5
    P = 0
    time_limit = 1800

    env = BikeNet(N, R, A, Q, t_repair, P, time_limit)

    RL = Train(A, 2 * A,
                   learning_rate=0.01,
                   reward_decay=0.9,
                   e_greedy=0.9,
                   replace_target_iter=200,
                   memory_size=2000,
                   # output_graph=True
               )
    output = simulate()
    #print(output)
    # RL.plot_cost()

## Raw Main Function

In [ ]:
# from maze_env import Maze
# from RL_brain import DeepQNetwork


def run_maze():
    step = 0
    for episode in range(300):
        # initial observation
        observation = env.reset()

        while True:
            # fresh env
            env.render()

            # RL choose action based on observation
            action = RL.choose_action(observation)

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            RL.store_transition(observation, action, reward, observation_)

            if (step > 200) and (step % 5 == 0):
                RL.learn()

            # swap observation
            observation = observation_

            # break while loop when end of this episode
            if done:
                break
            step += 1

    # end of game
    print('game over')
    env.destroy()


if __name__ == "__main__":
    # maze game
    env = Maze()
    RL = DeepQNetwork(env.n_actions, env.n_features,
                      learning_rate=0.01,
                      reward_decay=0.9,
                      e_greedy=0.9,
                      replace_target_iter=200,
                      memory_size=2000,
                      # output_graph=True
                      )
    env.after(100, run_maze)
    env.mainloop()
    RL.plot_cost()

## Class Maze

In [17]:
"""
Reinforcement learning maze example.
Red rectangle:          explorer.
Black rectangles:       hells       [reward = -1].
Yellow bin circle:      paradise    [reward = +1].
"""
import numpy as np
import time
import sys
if sys.version_info.major == 2:
    import Tkinter as tk
else:
    import tkinter as tk

UNIT = 40   # pixels
MAZE_H = 4  # grid height
MAZE_W = 4  # grid width


class Maze(tk.Tk, object):
    def __init__(self):
        super(Maze, self).__init__()
        self.action_space = ['u', 'd', 'l', 'r']
        self.n_actions = len(self.action_space)
        self.n_features = 2
        self.title('maze')
        self.geometry('{0}x{1}'.format(MAZE_H * UNIT, MAZE_H * UNIT))
        self._build_maze()

    def _build_maze(self):
        self.canvas = tk.Canvas(self, bg='white',
                           height=MAZE_H * UNIT,
                           width=MAZE_W * UNIT)

        # create grids
        for c in range(0, MAZE_W * UNIT, UNIT):
            x0, y0, x1, y1 = c, 0, c, MAZE_H * UNIT
            self.canvas.create_line(x0, y0, x1, y1)
        for r in range(0, MAZE_H * UNIT, UNIT):
            x0, y0, x1, y1 = 0, r, MAZE_W * UNIT, r
            self.canvas.create_line(x0, y0, x1, y1)

        # create origin
        origin = np.array([20, 20])

        # hell
        hell1_center = origin + np.array([UNIT * 2, UNIT])
        self.hell1 = self.canvas.create_rectangle(
            hell1_center[0] - 15, hell1_center[1] - 15,
            hell1_center[0] + 15, hell1_center[1] + 15,
            fill='black')
        # hell
        # hell2_center = origin + np.array([UNIT, UNIT * 2])
        # self.hell2 = self.canvas.create_rectangle(
        #     hell2_center[0] - 15, hell2_center[1] - 15,
        #     hell2_center[0] + 15, hell2_center[1] + 15,
        #     fill='black')

        # create oval
        oval_center = origin + UNIT * 2
        self.oval = self.canvas.create_oval(
            oval_center[0] - 15, oval_center[1] - 15,
            oval_center[0] + 15, oval_center[1] + 15,
            fill='yellow')

        # create red rect
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')

        # pack all
        self.canvas.pack()

    def reset(self):
        self.update()
        time.sleep(0.1)
        self.canvas.delete(self.rect)
        origin = np.array([20, 20])
        self.rect = self.canvas.create_rectangle(
            origin[0] - 15, origin[1] - 15,
            origin[0] + 15, origin[1] + 15,
            fill='red')
        # return observation
        return (np.array(self.canvas.coords(self.rect)[:2]) - np.array(self.canvas.coords(self.oval)[:2]))/(MAZE_H*UNIT)

    def step(self, action):
        s = self.canvas.coords(self.rect)
        base_action = np.array([0, 0])
        if action == 0:   # up
            if s[1] > UNIT:
                base_action[1] -= UNIT
        elif action == 1:   # down
            if s[1] < (MAZE_H - 1) * UNIT:
                base_action[1] += UNIT
        elif action == 2:   # right
            if s[0] < (MAZE_W - 1) * UNIT:
                base_action[0] += UNIT
        elif action == 3:   # left
            if s[0] > UNIT:
                base_action[0] -= UNIT

        self.canvas.move(self.rect, base_action[0], base_action[1])  # move agent

        next_coords = self.canvas.coords(self.rect)  # next state

        # reward function
        if next_coords == self.canvas.coords(self.oval):
            reward = 1
            done = True
        elif next_coords in [self.canvas.coords(self.hell1)]:
            reward = -1
            done = True
        else:
            reward = 0
            done = False
        s_ = (np.array(next_coords[:2]) - np.array(self.canvas.coords(self.oval)[:2]))/(MAZE_H*UNIT)
        return s_, reward, done

    def render(self):
        # time.sleep(0.01)
        self.update()

## Double DQN

In [ ]:
"""
The double DQN based on this paper: https://arxiv.org/abs/1509.06461
View more on my tutorial page: https://morvanzhou.github.io/tutorials/
Using:
Tensorflow: 1.0
gym: 0.8.0
"""

import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.set_random_seed(1)


class DoubleDQN:
    def __init__(
            self,
            n_actions,
            n_features,
            learning_rate=0.005,
            reward_decay=0.9,
            e_greedy=0.9,
            replace_target_iter=200,
            memory_size=3000,
            batch_size=32,
            e_greedy_increment=None,
            output_graph=False,
            double_q=True,
            sess=None,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max

        self.double_q = double_q    # decide to use double q or not

        self.learn_step_counter = 0
        self.memory = np.zeros((self.memory_size, n_features*2+2))
        self._build_net()
        t_params = tf.get_collection('target_net_params')
        e_params = tf.get_collection('eval_net_params')
        self.replace_target_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        if sess is None:
            self.sess = tf.Session()
            self.sess.run(tf.global_variables_initializer())
        else:
            self.sess = sess
        if output_graph:
            tf.summary.FileWriter("logs/", self.sess.graph)
        self.cost_his = []

    def _build_net(self):
        def build_layers(s, c_names, n_l1, w_initializer, b_initializer):
            with tf.variable_scope('l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], initializer=w_initializer, collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], initializer=b_initializer, collections=c_names)
                l1 = tf.nn.relu(tf.matmul(s, w1) + b1)

            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], initializer=w_initializer, collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], initializer=b_initializer, collections=c_names)
                out = tf.matmul(l1, w2) + b2
            return out
        # ------------------ build evaluate_net ------------------
        self.s = tf.placeholder(tf.float32, [None, self.n_features], name='s')  # input
        self.q_target = tf.placeholder(tf.float32, [None, self.n_actions], name='Q_target')  # for calculating loss

        with tf.variable_scope('eval_net'):
            c_names, n_l1, w_initializer, b_initializer = \
                ['eval_net_params', tf.GraphKeys.GLOBAL_VARIABLES], 20, \
                tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)  # config of layers

            self.q_eval = build_layers(self.s, c_names, n_l1, w_initializer, b_initializer)

        with tf.variable_scope('loss'):
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval))
        with tf.variable_scope('train'):
            self._train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)

        # ------------------ build target_net ------------------
        self.s_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')    # input
        with tf.variable_scope('target_net'):
            c_names = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES]

            self.q_next = build_layers(self.s_, c_names, n_l1, w_initializer, b_initializer)

    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        transition = np.hstack((s, [a, r], s_))
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition
        self.memory_counter += 1

    def choose_action(self, observation):
        observation = observation[np.newaxis, :]
        actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
        action = np.argmax(actions_value)

        if not hasattr(self, 'q'):  # record action value it gets
            self.q = []
            self.running_q = 0
        self.running_q = self.running_q*0.99 + 0.01 * np.max(actions_value)
        self.q.append(self.running_q)

        if np.random.uniform() > self.epsilon:  # choosing action
            action = np.random.randint(0, self.n_actions)
        return action

    def learn(self):
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.replace_target_op)
            print('\ntarget_params_replaced\n')

        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        q_next, q_eval4next = self.sess.run(
            [self.q_next, self.q_eval],
            feed_dict={self.s_: batch_memory[:, -self.n_features:],    # next observation
                       self.s: batch_memory[:, -self.n_features:]})    # next observation
        q_eval = self.sess.run(self.q_eval, {self.s: batch_memory[:, :self.n_features]})

        q_target = q_eval.copy()

        batch_index = np.arange(self.batch_size, dtype=np.int32)
        eval_act_index = batch_memory[:, self.n_features].astype(int)
        reward = batch_memory[:, self.n_features + 1]

        if self.double_q:
            max_act4next = np.argmax(q_eval4next, axis=1)        # the action that brings the highest value is evaluated by q_eval
            selected_q_next = q_next[batch_index, max_act4next]  # Double DQN, select q_next depending on above actions
        else:
            selected_q_next = np.max(q_next, axis=1)    # the natural DQN

        q_target[batch_index, eval_act_index] = reward + self.gamma * selected_q_next

        _, self.cost = self.sess.run([self._train_op, self.loss],
                                     feed_dict={self.s: batch_memory[:, :self.n_features],
                                                self.q_target: q_target})
        self.cost_his.append(self.cost)

        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1



In [ ]:
"""
Double DQN & Natural DQN comparison,
The Pendulum example.
View more on my tutorial page: https://morvanzhou.github.io/tutorials/
Using:
Tensorflow: 1.0
gym: 0.8.0
"""


import gym
#from RL_brain import DoubleDQN
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


env = gym.make('Pendulum-v0')
env = env.unwrapped
env.seed(1)
MEMORY_SIZE = 3000
ACTION_SPACE = 11

sess = tf.Session()
with tf.variable_scope('Natural_DQN'):
    natural_DQN = DoubleDQN(
        n_actions=ACTION_SPACE, n_features=3, memory_size=MEMORY_SIZE,
        e_greedy_increment=0.001, double_q=False, sess=sess
    )

with tf.variable_scope('Double_DQN'):
    double_DQN = DoubleDQN(
        n_actions=ACTION_SPACE, n_features=3, memory_size=MEMORY_SIZE,
        e_greedy_increment=0.001, double_q=True, sess=sess, output_graph=True)

sess.run(tf.global_variables_initializer())


def train(RL):
    total_steps = 0
    observation = env.reset()
    while True:
        # if total_steps - MEMORY_SIZE > 8000: env.render()

        action = RL.choose_action(observation)

        f_action = (action-(ACTION_SPACE-1)/2)/((ACTION_SPACE-1)/4)   # convert to [-2 ~ 2] float actions
        observation_, reward, done, info = env.step(np.array([f_action]))

        reward /= 10     # normalize to a range of (-1, 0). r = 0 when get upright
        # the Q target at upright state will be 0, because Q_target = r + gamma * Qmax(s', a') = 0 + gamma * 0
        # so when Q at this state is greater than 0, the agent overestimates the Q. Please refer to the final result.

        RL.store_transition(observation, action, reward, observation_)

        if total_steps > MEMORY_SIZE:   # learning
            RL.learn()

        if total_steps - MEMORY_SIZE > 3:   # stop game
            break

        observation = observation_
        total_steps += 1
    return RL.q

q_natural = train(natural_DQN)
q_double = train(double_DQN)

plt.plot(np.array(q_natural), c='r', label='natural')
plt.plot(np.array(q_double), c='b', label='double')
plt.legend(loc='best')
plt.ylabel('Q eval')
plt.xlabel('training steps')
plt.grid()
plt.show()

In [ ]:
# 程序仿真部分

import numpy as np
import pandas as pd
from numba import jit

class Area():
    def __init__(self, n, a_id):
        self.a_id = a_id
        self.normal_bike = n
        self.broken_bike = 0

    def move(self):
        self.normal_bike -= 1
        self.broken_bike += 1


def binaryInsert(target, events):
    for event in events:
        if event >= target[-1]:
            target.append(event)
        else:
            l, mid, r = 0, int(len(target) / 2), len(target) - 1
            while 1:
                if r - l == 1:
                    target.insert(r, event)
                    break
                else:
                    if event > target[mid]:
                        l = mid
                        mid = int((r + l) / 2)
                    else:
                        r = mid
                        mid = int((r + l) / 2)


def simulate(N, R, A, Q, P, t_limit):
    '''
    N: initial number of bikes in the network
    R: rates of arriving and departing for each node
    A: number of Areas
    Q: matrix of tansitition probability
    P: policy of dealing with broken bikes

    T: system clock
    scheduler: stack of upcoming events,[t, event_type, area], 1 for customer arrival, 2 for bike arrival
    
    stat: record of system state parameters
    loss: record of customer loss
    '''

    # initiate
    T = 0
    #stat, S = pd.DataFrame(columns=['type', 'place', 't']), 0
    loss, L = pd.DataFrame(columns=['place', 't']), 0
    broken, B = pd.DataFrame(columns=['place', 'ng', 'nb', 't']), 0

    # initiation of instances of Area and scheduler
    scheduler = []
    a = []
    for i in range(A):
        a.append(Area(N / A, i))
        scheduler.append([np.random.exponential(1/R.loc[i].cus_arr), 1, a[i]])
    scheduler.sort()

    # system running
    while T < t_limit:
        
        T = scheduler[0][0]
        if scheduler[0][1] == 1:
            #stat.loc[S], S = [scheduler[0][1], scheduler[0][2].a_id, T], S+1
            if scheduler[0][2].normal_bike == 0:
                # this is a loss
                loss.loc[L], L = [scheduler[0][2].a_id, T], L+1
                event = [T + np.random.exponential(1/R.loc[scheduler[0][2].a_id].cus_arr), 1, scheduler[0][2]]
                binaryInsert(scheduler, [event])
            else:
                target = np.random.choice(np.arange(A+1), 1, p=Q[scheduler[0][2].a_id])
                if target == A:
                    broken.loc[B], B = [scheduler[0][2].a_id, scheduler[0][2].normal_bike, scheduler[0][2].broken_bike, T], B+1
                    scheduler[0][2].move()
                    continue
                else:
                    scheduler[0][2].normal_bike -= 1
                    event1 = [T + np.random.exponential(1/R.loc[scheduler[0][2].a_id].ride), 2, a[target[0]]]
                    event2 = [T + np.random.exponential(1/R.loc[scheduler[0][2].a_id].cus_arr), 1, scheduler[0][2]]
                    binaryInsert(scheduler, [event1, event2])
        else:
            #stat.loc[S], S = [scheduler[0][1], scheduler[0][2].a_id, T], S+1
            scheduler[0][2].normal_bike += 1
        scheduler.pop(0)
        
    #return stat
    return loss, broken

if __name__ == '__main__':
    np.random.seed(1)
    N = 360
    A = 9
    R = pd.DataFrame({'cus_arr': [5] * A, 'ride': [10] * A}, index=range(A))
    Q = [np.random.rand(A+1) for i in range(A)]
    Q = [q / sum(q) for q in Q]
    #Q = [[0,0.9,0.1], [0.9,0,0.1]]
    P = 0
    time_limit = 6000

    result = simulate(N, R, A, Q, P, time_limit)

